# Marlowe "Swap" Contract Run on Mainnet Using a Reference Script

This transcript documents the first use of a reference script for a Marlowe transaction on `mainnet`.

The scenario is that John Fletcher trades 300 `Globe` tokens with Thomas Kyd for 500 `Swan` tokens.

1. Create the Marlowe contract instance: [72482ce24c88a9b187ab58e34eb1ed1681f4b450557cc13654458e86fd8ff15d](https://cardanoscan.io/transaction/72482ce24c88a9b187ab58e34eb1ed1681f4b450557cc13654458e86fd8ff15d)
2. Party A deposits tokens:  [99c4b7dceffaa9daa3613efe89155976ca42010077d40287ff6e779217849057](https://cardanoscan.io/transaction/99c4b7dceffaa9daa3613efe89155976ca42010077d40287ff6e779217849057)
3. Party B deposits tokens to complete the swap: [4e8cd7a65ddc533e974a499ef6efb108177851eadc818ccd30f44be936592295](https://cardanoscan.io/transaction/4e8cd7a65ddc533e974a499ef6efb108177851eadc818ccd30f44be936592295)

You can view all of the Marlowe transaction at the address for the current version of the Marlowe validator: [addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf](https://cardanoscan.io/address/addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf)

## Preliminaries

Record version numbers

In [4]:
marlowe-cli --version

marlowe-cli 0.0.8.3 @ 3709087ea19ba42b60ffc302373fa28cc9d368dc


In [4]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [5]:
git rev-parse HEAD

25f6808de7aca1ab3d47db52fb4b66cae4228618


### Setup the faucet.

Set the location of keys.

In [11]:
TREASURY=treasury

Set the faucet.

In [16]:
FAUCET_SKEY_FILE=$TREASURY/payment.skey
FAUCET_ADDRESS=$(cat $TREASURY/payment.mainnet.address)
echo "$FAUCET_ADDRESS"

addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h


### Select Network

In [37]:
export CARDANO_TESTNET_MAGIC=764824073
export CARDANO_NODE_SOCKET_PATH=node.socket

Compute the slot-to-time parameters.

In [29]:
SLOT_LENGTH=$(marlowe-cli util slotting | jq .scSlotLength)
SLOT_OFFSET=$(marlowe-cli util slotting | jq .scSlotZeroTime)

### Tip of the Blockchain

In [30]:
TIP=$(cardano-cli query tip --mainnet | jq '.slot')
NOW="$((TIP*SLOT_LENGTH+SLOT_OFFSET))"
HOUR="$((3600*1000))"

The tip of the blockchain:

In [31]:
echo $TIP

72762292


The current POSIX time implies that the tip of the blockchain should be slightly before the following slot:

In [32]:
echo $(((1000 * $(date -u +%s) - SLOT_OFFSET) / SLOT_LENGTH))

72762308


Tests main fail if this is not the case.

### Publish reference script for Marlowe onto `mainnet`

Publish the Marlowe semantics validator as a reference script.

In [17]:
marlowe-cli transaction publish --required-signer "$FAUCET_SKEY_FILE" \
                                --change-address "$FAUCET_ADDRESS" \
                                --at-address addr1qypn5mwucvjm0pkfwemjus8n7glep8927sj76dv9a3xnq4j7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqfk0yl3 \
                                --out-file /dev/null \
                                --submit 600


Fee: Lovelace 846357
Size: 15461 / 16384 = 94%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%

Marlowe script published at address: addr1qypn5mwucvjm0pkfwemjus8n7glep8927sj76dv9a3xnq4j7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqfk0yl3

Marlowe script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Marlowe ref script UTxO min ADA: Lovelace 12555030

Payout script published at address: addr1qypn5mwucvjm0pkfwemjus8n7glep8927sj76dv9a3xnq4j7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqfk0yl3

Payout script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"

Payout ref script UTxO min ADA: Lovelace 12555030


Check that the Marlowe semantics validator was published.

In [19]:
marlowe-cli transaction find-published --at-address addr1qypn5mwucvjm0pkfwemjus8n7glep8927sj76dv9a3xnq4j7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqfk0yl3


Searching for reference script at address: addr1qypn5mwucvjm0pkfwemjus8n7glep8927sj76dv9a3xnq4j7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqfk0yl3

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr1qypn5mwucvjm0pkfwemjus8n7glep8927sj76dv9a3xnq4j7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqfk0yl3

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "9dc19f70488336dbf3d68388fcfb2e3f7a3d7ed6358422f37b10aa6625246c71#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "9dc19f70488336dbf3d68388fcfb2e3f7a3d7ed6358422f37b10aa6625246c71#2"
    }
}


Set the location of the node socket.

In [7]:
export CARDANO_NODE_SOCKET_PATH=node.socket

### Participants

#### The First Party

In [33]:
PARTY_A_PREFIX="$TREASURY/john-fletcher"
PARTY_A_PAYMENT_SKEY="$PARTY_A_PREFIX".skey
PARTY_A_PAYMENT_VKEY="$PARTY_A_PREFIX".vkey

Create the first party's keys, if necessary.

In [39]:
if [[ ! -e "$PARTY_A_PAYMENT_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_A_PAYMENT_SKEY"      \
                              --verification-key-file "$PARTY_A_PAYMENT_VKEY"
fi
PARTY_A_ADDRESS=$(cardano-cli address build --mainnet --payment-verification-key-file "$PARTY_A_PAYMENT_VKEY" )
PARTY_A_PUBKEYHASH=$(cardano-cli address key-hash --payment-verification-key-file "$PARTY_A_PAYMENT_VKEY")

Address of the first party and its public key hash:

In [40]:
echo "Address: $PARTY_A_ADDRESS"
echo "Public key hash: $PARTY_A_PUBKEYHASH"

Address: addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx
Public key hash: 1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf


Fund the first party's address.

In [41]:
marlowe-cli util faucet --out-file /dev/null                      \
                        --submit 600                              \
                        --lovelace 100000000                      \
                        --faucet-address "$FAUCET_ADDRESS"        \
                        --required-signer "$FAUCET_SKEY_FILE"     \
                        "$PARTY_A_ADDRESS"

TxId "c5bc6e4466d5496bc7578f53bfe905f3324337f54239c42aff4a1dd934af6e67"


The first party mints their tokens for the swap.

In [42]:
MINT_EXPIRES=$((TIP + 1000000))

In [43]:
TOKEN_NAME_A=Globe
AMOUNT_A=300
CURRENCY_SYMBOL_A=$(
marlowe-cli util mint --required-signer "$PARTY_A_PAYMENT_SKEY" \
                      --change-address  "$PARTY_A_ADDRESS"      \
                      --count "$AMOUNT_A"                       \
                      --expires "$MINT_EXPIRES"                 \
                      --out-file /dev/null                      \
                      --submit=600                              \
                      "$TOKEN_NAME_A"                           \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                            \
)
TOKEN_A="$CURRENCY_SYMBOL_A.$TOKEN_NAME_A"

They have the following UTxOs in their wallet:

In [44]:
marlowe-cli util clean --required-signer "$PARTY_A_PAYMENT_SKEY" \
                       --change-address "$PARTY_A_ADDRESS"       \
                       --out-file /dev/null                      \
                       --submit=600                              \
> /dev/null
cardano-cli query utxo --mainnet --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
11ad1c97e268f6de6b708cc83bd98b79534db1191e2a699f46f5e0c800bdc6bc     0        97651266 lovelace + TxOutDatumNone
11ad1c97e268f6de6b708cc83bd98b79534db1191e2a699f46f5e0c800bdc6bc     1        2000000 lovelace + 300 a7f7e57db27c9e2f80c205ccb30f73e57f0ee8fc21aff7b86b5daf78.476c6f6265 + TxOutDatumNone


The first party will trade the following in the swap:

In [45]:
echo "Amount: $AMOUNT_A"
echo "Token: $TOKEN_A"

Amount: 300
Token: a7f7e57db27c9e2f80c205ccb30f73e57f0ee8fc21aff7b86b5daf78.Globe


#### The Second Party

In [46]:
PARTY_B_PREFIX="$TREASURY/thomas-kyd"
PARTY_B_PAYMENT_SKEY="$PARTY_B_PREFIX".skey
PARTY_B_PAYMENT_VKEY="$PARTY_B_PREFIX".vkey

Create the second party's keys, if necessary.

In [47]:
if [[ ! -e "$PARTY_B_PAYMENT_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_B_PAYMENT_SKEY"      \
                              --verification-key-file "$PARTY_B_PAYMENT_VKEY"
fi
PARTY_B_ADDRESS=$(cardano-cli address build --mainnet --payment-verification-key-file "$PARTY_B_PAYMENT_VKEY" )
PARTY_B_PUBKEYHASH=$(cardano-cli address key-hash --payment-verification-key-file "$PARTY_B_PAYMENT_VKEY")

Address of the second party and its public key hash:

In [48]:
echo "Address: $PARTY_B_ADDRESS"
echo "Public key hash: $PARTY_B_PUBKEYHASH"

Address: addr1v87n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlchhlqt3
Public key hash: fd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff


Fund the second party's address.

In [50]:
marlowe-cli util faucet --out-file /dev/null                  \
                        --submit 600                          \
                        --lovelace 100000000                  \
                        --faucet-address "$FAUCET_ADDRESS"    \
                        --required-signer "$FAUCET_SKEY_FILE" \
                        "$PARTY_B_ADDRESS"

TxId "20318a3f543ac67ac4547707299972a5343d0a0fed008695d453a86ac377752f"


The second party mints their tokens for the swap.

In [51]:
TOKEN_NAME_B=Swan
AMOUNT_B=500
CURRENCY_SYMBOL_B=$(
marlowe-cli util mint --required-signer "$PARTY_B_PAYMENT_SKEY" \
                      --change-address  "$PARTY_B_ADDRESS"      \
                      --count "$AMOUNT_B"                       \
                      --expires "$MINT_EXPIRES"                 \
                      --out-file /dev/null                      \
                      --submit=600                              \
                      "$TOKEN_NAME_B"                           \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                            \
)
TOKEN_B="$CURRENCY_SYMBOL_B.$TOKEN_NAME_B"

They have the following UTxOs in their wallet:

In [52]:
marlowe-cli util clean --required-signer "$PARTY_B_PAYMENT_SKEY" \
                       --change-address "$PARTY_B_ADDRESS"       \
                       --out-file /dev/null                      \
                       --submit=600                              \
> /dev/null
cardano-cli query utxo --mint "$CARDANO_TESTNET_MAGIC" --address "$PARTY_B_ADDRESS"

Invalid option `--mint'

Usage: cardano-cli query utxo 
            [ --shelley-mode
            | --byron-mode [--epoch-slots NATURAL]
            | --cardano-mode [--epoch-slots NATURAL]
            ]
            (--whole-utxo | (--address ADDRESS) | (--tx-in TX-IN))
            (--mainnet | --testnet-magic NATURAL)
            [--out-file FILE]

  Get a portion of the current UTxO: by tx in, by address or the whole.


: 1

The second party will trade the following in the swap:

In [53]:
echo "Amount: $AMOUNT_B"
echo "Token: $TOKEN_B"

Amount: 500
Token: ecc8ad61b973946ee1cc666b259acabb3edf38a73f1b8779d93ba28a.Swan


## The Contract

The initial Ada in the contract.

In [54]:
MINIMUM_ADA=3000000

In [55]:
PARTY_A_TIMEOUT=$((NOW+12*HOUR))
PARTY_B_TIMEOUT=$((NOW+24*HOUR))

The timeout for the first party:

In [56]:
echo $(date -u -R -d @$((PARTY_A_TIMEOUT/1000)))

Wed, 28 Sep 2022 13:29:43 +0000


The timeout for the second party:

In [57]:
echo $(date -u -R -d @$((PARTY_B_TIMEOUT/1000)))

Thu, 29 Sep 2022 01:29:43 +0000


We create the contract for the previously specified parameters.

In [60]:
marlowe-cli template swap --minimum-ada "$MINIMUM_ADA"       \
                          --a-party "$PARTY_A_ADDRESS" \
                          --a-token "$TOKEN_A"               \
                          --a-amount "$AMOUNT_A"             \
                          --a-timeout "$PARTY_A_TIMEOUT"     \
                          --b-party "$PARTY_B_ADDRESS" \
                          --b-token "$TOKEN_B"               \
                          --b-amount "$AMOUNT_B"             \
                          --b-timeout "$PARTY_B_TIMEOUT"     \
                          --out-contract-file tx-1.contract  \
                          --out-state-file    tx-1.state

## Transaction 1. Create the Contract by Providing the Minimum ADA.

First we create a `.marlowe` file that contains the initial information needed to run the contract. The bare size and cost of the script provide a lower bound on the resources that running it will require.

In [61]:
marlowe-cli run initialize --contract-file tx-1.contract \
                           --state-file    tx-1.state    \
                           --out-file      tx-1.marlowe  \
                           --print-stats


Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


In particular, we can extract the contract's address from the `.marlowe` file.

In [62]:
CONTRACT_ADDRESS=$(jq -r '.tx.marloweValidator.address' tx-1.marlowe)

The Marlowe contract resides at the following address:

In [63]:
echo $CONTRACT_ADDRESS

addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf


The first party submits the transaction along with the minimum Ada required for the contract's initial state. Submitting with the `--print-stats` switch reveals the network fee for the contract, the size of the transaction, and the execution requirements, relative to the protocol limits.

In [64]:
TX_1=$(
marlowe-cli run auto-execute --required-signer "$PARTY_A_PAYMENT_SKEY" \
                             --marlowe-out-file tx-1.marlowe           \
                             --change-address "$PARTY_A_ADDRESS"       \
                             --out-file tx-1.raw                       \
                             --print-stats                             \
                             --submit 600                              \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
)


Fee: Lovelace 214385
Size: 995 / 16384 = 6%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%


The contract received the minimum Ada from the first party in the following transaction:

In [65]:
echo $TX_1

72482ce24c88a9b187ab58e34eb1ed1681f4b450557cc13654458e86fd8ff15d


The following UTxOs are at the contract address:

In [68]:
cardano-cli query utxo --mainnet --address "$CONTRACT_ADDRESS" | sed -n -e "1p;2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
72482ce24c88a9b187ab58e34eb1ed1681f4b450557cc13654458e86fd8ff15d     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "943c96772987263bcc561c8e97803b2fd69d7eea25cfd241adec088ddfc4f937"


Here are the UTxOs at the first party address:

In [69]:
cardano-cli query utxo --mainnet  --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
11ad1c97e268f6de6b708cc83bd98b79534db1191e2a699f46f5e0c800bdc6bc     1        2000000 lovelace + 300 a7f7e57db27c9e2f80c205ccb30f73e57f0ee8fc21aff7b86b5daf78.476c6f6265 + TxOutDatumNone
72482ce24c88a9b187ab58e34eb1ed1681f4b450557cc13654458e86fd8ff15d     0        94436881 lovelace + TxOutDatumNone


## Transaction 2. First Party Deposits Tokens into the Contract.

First we compute the Marlowe input required to deposit the tokens.

In [70]:
marlowe-cli run prepare --marlowe-file tx-1.marlowe                \
                        --deposit-account "$PARTY_A_ADDRESS" \
                        --deposit-party "$PARTY_A_ADDRESS"   \
                        --deposit-token "$TOKEN_A"                 \
                        --deposit-amount "$AMOUNT_A"               \
                        --invalid-before "$NOW"                    \
                        --invalid-hereafter "$((NOW+4*HOUR))"      \
                        --out-file tx-2.marlowe                    \
                        --print-stats


Datum size: 657


Now the first party submits the transaction that deposits their tokens.

In [71]:
TX_2=$(
marlowe-cli run auto-execute --marlowe-in-file tx-1.marlowe            \
                             --tx-in-marlowe "$TX_1#1"                 \
                             --required-signer "$PARTY_A_PAYMENT_SKEY" \
                             --marlowe-out-file tx-2.marlowe           \
                             --change-address "$PARTY_A_ADDRESS"       \
                             --out-file tx-2.raw                       \
                             --print-stats                             \
                             --submit=600                              \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
)


Fee: Lovelace 669379
Size: 1977 / 16384 = 12%
Execution units:
  Memory: 5252696 / 14000000 = 37%
  Steps: 1444845510 / 10000000000 = 14%


The contract received the deposit of the first party's token for the swap in the following transaction:

In [72]:
echo $TX_2

99c4b7dceffaa9daa3613efe89155976ca42010077d40287ff6e779217849057


The following UTxOs are at the contract address:

In [73]:
cardano-cli query utxo --mainnet --address "$CONTRACT_ADDRESS" | sed -n -e "1p;2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
99c4b7dceffaa9daa3613efe89155976ca42010077d40287ff6e779217849057     1        3000000 lovelace + 300 a7f7e57db27c9e2f80c205ccb30f73e57f0ee8fc21aff7b86b5daf78.476c6f6265 + TxOutDatumHash ScriptDataInBabbageEra "e2072a520929b3ea43ada85b5047df082ddf24f2c3fb80628c687886e11a9343"


Here are the UTxOs at the first party address:

In [75]:
cardano-cli query utxo --mainnet --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
99c4b7dceffaa9daa3613efe89155976ca42010077d40287ff6e779217849057     0        95767502 lovelace + TxOutDatumNone


## Transaction 3. The Second Party Deposits their Tokens to Complete the Swap.

In [76]:
marlowe-cli run prepare --marlowe-file tx-2.marlowe                \
                        --deposit-account "$PARTY_B_ADDRESS" \
                        --deposit-party "$PARTY_B_ADDRESS"   \
                        --deposit-token "$TOKEN_B"                 \
                        --deposit-amount "$AMOUNT_B"               \
                        --invalid-before "$NOW"                    \
                        --invalid-hereafter "$((NOW+4*HOUR))"      \
                        --out-file tx-3.marlowe                    \
                        --print-stats


Datum size: 30
Payment 1
  Acccount: "\"addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx\""
  Payee: Party "\"addr1v87n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlchhlqt3\""
  Ada: Lovelace {getLovelace = 0}
  a7f7e57db27c9e2f80c205ccb30f73e57f0ee8fc21aff7b86b5daf78."Globe": 300
Payment 2
  Acccount: "\"addr1v87n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlchhlqt3\""
  Payee: Party "\"addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx\""
  Ada: Lovelace {getLovelace = 0}
  ecc8ad61b973946ee1cc666b259acabb3edf38a73f1b8779d93ba28a."Swan": 500
Payment 3
  Acccount: "\"addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx\""
  Payee: Party "\"addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx\""
  Ada: Lovelace {getLovelace = 3000000}


Now the second party can submit a transaction that deposits their tokens and completes the swap.

In [77]:
TX_3=$(
marlowe-cli run auto-execute --marlowe-in-file tx-2.marlowe            \
                             --tx-in-marlowe "$TX_2#1"                 \
                             --required-signer "$PARTY_B_PAYMENT_SKEY" \
                             --marlowe-out-file tx-3.marlowe           \
                             --change-address "$PARTY_B_ADDRESS"       \
                             --out-file tx-3.raw                       \
                             --print-stats                             \
                             --submit=600                              \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
)


Fee: Lovelace 815462
Size: 1298 / 16384 = 7%
Execution units:
  Memory: 7615252 / 14000000 = 54%
  Steps: 1994618836 / 10000000000 = 19%


The closing of the contract paid the swapped tokens to the parties in the transaction:

In [81]:
echo $TX_3

4e8cd7a65ddc533e974a499ef6efb108177851eadc818ccd30f44be936592295


There is no UTxO at the contract address:

In [78]:
cardano-cli query utxo --mainnet --address "$CONTRACT_ADDRESS" | sed -n -e "1p;2p;/$TX_1/p;/$TX_2/p;/$TX_3/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Here are the UTxOs at the first party's address:

In [79]:
cardano-cli query utxo --mainnet --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
4e8cd7a65ddc533e974a499ef6efb108177851eadc818ccd30f44be936592295     1        3000000 lovelace + 500 ecc8ad61b973946ee1cc666b259acabb3edf38a73f1b8779d93ba28a.5377616e + TxOutDatumNone
99c4b7dceffaa9daa3613efe89155976ca42010077d40287ff6e779217849057     0        95767502 lovelace + TxOutDatumNone


Here are the UTxOs at the second party's address:

In [80]:
cardano-cli query utxo --mainnet --address "$PARTY_B_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
4e8cd7a65ddc533e974a499ef6efb108177851eadc818ccd30f44be936592295     0        97797226 lovelace + TxOutDatumNone
4e8cd7a65ddc533e974a499ef6efb108177851eadc818ccd30f44be936592295     2        1038710 lovelace + 300 a7f7e57db27c9e2f80c205ccb30f73e57f0ee8fc21aff7b86b5daf78.476c6f6265 + TxOutDatumNone


**Voilà!**